In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import markdown
from markdown.extensions import codehilite


In [17]:
df = pd.read_csv('../data_files/raw/summaries_V0903_for_humanstudy_detail.csv')

In [18]:
df.model.unique()

array(['deepseek-chat', 'gemini-2.5-pro', 'gpt-5', 'qwen3-32b',
       'web-rev-claude-opus-4-20250514'], dtype=object)

In [19]:
df = df.sample(frac=1).reset_index(drop=True)

In [20]:
df

,id,topic,question,summary,model,comment_num,comments,num_samples_group,sample_id,dataset_name,source_path
0,4a730f0d1d502f37,Binary-Health-Care-Policy,Do you support the government provide basic he...,Overall summary of comments\n\n- Sentiment sna...,gpt-5,70,"0: Yes, I think government based healthcare is...",70,1,Binary-Health-Care-Policy,results/human_judgement/gpt-5/70/Binary-Health...
1,9bb23a8098a7f00a,Binary-Vaccination-Policy,Do you support the government having the autho...,Looking at the range of opinions on government...,web-rev-claude-opus-4-20250514,30,0: I do not. I think that people can be highly...,30,3,Binary-Vaccination-Policy,results/human_judgement/web-rev-claude-opus-4-...
2,a6afa7fd357f4dad,Binary-Refugee-Policies,Do you support the government accepting more r...,Here is a summary of the comments provided:\n\...,gemini-2.5-pro,30,"0: I am pretty conflicted, but I think overall...",30,2,Binary-Refugee-Policies,results/human_judgement/gemini-2.5-pro/30/Bina...
3,15e5a1375766919a,Openqa-AI-changes-human-life,How has AI changed your life? Please answer br...,"Overall, the comments reflect a complex and mu...",deepseek-chat,50,0: Al changed my life by making my daily tasks...,50,1,Openqa-AI-changes-human-life,results/human_judgement/deepseek-chat/50/Openq...
4,35f266febb6ebd81,Openqa-Trump-cutting-funding,What are your thoughts on Trump’s decision to ...,Overall summary of comments:\n- Overall sentim...,gpt-5,50,0: I think it's ridiculous. Education is liter...,50,1,Openqa-Trump-cutting-funding,results/human_judgement/gpt-5/50/Openqa-Trump-...
...,...,...,...,...,...,...,...,...,...,...,...
745,093c525905a8bb21,Binary-Vaccination-Policy,Do you support the government having the autho...,"Based on the comments provided, perspectives o...",deepseek-chat,90,0: I do not. I think that people can be highly...,90,2,Binary-Vaccination-Policy,results/human_judgement/deepseek-chat/90/Binar...
746,c51afc06adf76e19,Openqa-AI-changes-human-life,How has AI changed your life? Please answer br...,"Overall, the comments portray a largely positi...",gpt-5,10,0: Al changed my life by making my daily tasks...,10,3,Openqa-AI-changes-human-life,results/human_judgement/gpt-5/10/Openqa-AI-cha...
747,2e0d2b716607a67b,Openqa-Trump-cutting-funding,What are your thoughts on Trump’s decision to ...,## Summary of Comments on Academic Funding Cut...,web-rev-claude-opus-4-20250514,90,0: I think it's ridiculous. Education is liter...,90,3,Openqa-Trump-cutting-funding,results/human_judgement/web-rev-claude-opus-4-...
748,38b84ade368fb29e,Openqa-Tipping-System,"What is your opinion on tipping, and if given ...",## Overall Summary of Comments on Tipping\n\nT...,web-rev-claude-opus-4-20250514,90,0: I would get rid of the tipping system entir...,90,1,Openqa-Tipping-System,results/human_judgement/web-rev-claude-opus-4-...


In [32]:


# Process the dataframe into the desired format
processed_data = []

for i, (_, row) in enumerate(df.iterrows()):
    raw_id = row['id']
    question = row['question']
    summary = row['summary']
    
    # Add a row for the question
    question_entry = {
        "id": f"{raw_id}_question",
        "raw_id": raw_id,
        "question": question,
        "text": '[Question]' + question.replace("\n", "<br>").replace(" Please answer briefly in 2–3 sentences.", "").replace("Please answer briefly in 1–2 sentences.", ""),
        "model": row['model'],
        "summary_length": row.get('summary_length', None)
    }
    processed_data.append(question_entry)
    
    # Add a row for the summary (convert markdown to HTML using markdown package)
    summary_html = markdown.markdown(summary, extensions=['extra', 'codehilite'])
    summary_entry = {
        "id": f"{raw_id}_summary",
        "raw_id": raw_id,
        "question": question,
        "text": "Below is a summary of people's opinions on the issue.<br><hr><br>" + summary_html,
        "model": row['model'],
        "summary_length": row.get('summary_length', None)
    }
    processed_data.append(summary_entry)

# Convert the processed data into a DataFrame
processed_df = pd.DataFrame(processed_data)


In [38]:
processed_df['text'].value_counts()[:10]

text
[Question]What is your opinion on the rapid update cycle of electronic products, especially smartphones?                                                            75
[Question]What is your opinion on internet influencers (e.g., streamers, bloggers, short video creators) increasingly becoming a recognized profession?             75
[Question]What are your thoughts on Trump’s decision to cut academic funding?                                                                                       75
[Question]How has AI changed your life?                                                                                                                             75
[Question]Do you support the government provide basic health insurance for everyone?                                                                                75
[Question]Do you support requiring real-name registration on social media platforms, where users must register and post under their real identity?              

In [39]:
processed_df[:20].to_csv('../data_files/processed/sum_humanstudy_v0903_pilot_20.csv', index=False)